In [1]:
from modules.input_gen import generate_prompt

prompt, output_filename = generate_prompt()

print(prompt)
run_idx = 0

A QR code, cleverly woven into a colorful modern art gallery, glowing theme, surrealism, 3D rendered


In [2]:
from modules.input_gen import generate_wifi_qr_string
from modules.input_gen import generate_qr_code
import qrcode

data = generate_wifi_qr_string("Vodafone-FD4C", "pxfy2x2gbq23p3hh")
inverted = False
output_dir = 'v2_code'

qr_params = {
    'data': data,
    'filename': 'qr_with_mask',
    'version': None, 
    'error_correction': qrcode.constants.ERROR_CORRECT_H,
    'box_size': 10,
    'border': 10,
    'inverted': inverted,
    'add_logo': False,
    'logo_path': 'logo.png',
    'logo_scale': 1.5,
    'central_mask_shape': 'circle',  #('circle', 'square', 'diamond')
    'central_mask_size': 20,
    'column_width': 20,
    'border_softness': 0
}
# Generate the QR code with custom parameters
qr_image, masks = generate_qr_code(**qr_params)

from modules.diffuser import run_diffusion_on_qr_code

# Parameters for add_noise_to_qr_code function
noise_params = {
    'masks': masks,
    'qr_noise_level': 0,
    'padding_noise_level': 0.5,
    'central_noise_level': 0.3,
    'noise_type': 'uniform',  #('gaussian', 'salt_pepper', 'speckle')
    'filename': 'noisy_qr_code',
    'inverted': inverted,
}

# Parameters for the diffusion model
diffusion_params = {
    'image_of_qr': qr_image,
    'prompt': prompt,
    'output_filename': f'{output_dir + output_filename}',
    'model_id': 'models/animerge_v23',
    'controlnet_model_id': 'DionTimmer/controlnet_qrcode-control_v1p_sd15',
    'strength': 0.9,
    'guidance_scale': 22,
    'controlnet_conditioning_scale': 2,
    'num_inference_steps': 50,
    'resolution': 512,
    'seed': 42,
    'noise_params' : noise_params,
    'verbose': True,
}

# Run the diffusion model on the noisy QR code
run_diffusion_on_qr_code(**diffusion_params)

#for padding_noise in [0.1, 0.2, 0.3, 0.4, 0.5]:
#    diffusion_params['noise_params']['padding_noise_level'] = padding_noise
#    # add number to output filename
#    diffusion_params['output_filename'] = output_dir + f'{padding_noise}' + output_filename
#    run_diffusion_on_qr_code(**diffusion_params)


run_idx += 1

TypeError: generate_qr_code() got an unexpected keyword argument 'central_mask_shape'

In [3]:
import itertools
import os
import random
import numpy as np
from modules.diffuser import run_diffusion_on_qr_code
from modules.input_gen import generate_qr_code
from modules.input_gen import generate_prompt
from modules.input_gen import generate_wifi_qr_string

# Define the ranges for each parameter
param_grid = {
    'strength': [0.9],  # Example values for strength
    'guidance_scale': [14, 20],  # Example values for guidance_scale
    #'controlnet_conditioning_scale': np.arange(1.1, 4.4, 0.2).round(1).tolist(),  # Example values for controlnet_conditioning_scale
    'controlnet_conditioning_scale': [2.2, 2.8],  
    'resolution': [1536],  # Example resolution
    #'seed': [random.randint(0, 100000) for _ in range(8)],  # Random seeds
    'seed': range(1),  # seeds
    'mask_logo': [120, 140],  # Example values for mask_logo
    'border': [10],  # Example values for border
    'noise_level': [0.5],  # Example values for noise_level
    'border_noise_level': [1],  # Example values for border_noise_level
    'center_noise_level': [1],  # Example values for center_noise_level
    'num_inference_steps': [20],  # Example values for num_inference_steps
    'inverted': [True],  # Inverted QR code colors
    'add_logo': [False]  # Add a logo to the QR code
}

# User-specified output directory
output_dir = "large_files_low_border"
#output_dir = "wifi_jonas"
os.makedirs(f'output_images/{output_dir}', exist_ok=True)

# Generate all combinations of parameters
keys, values = zip(*param_grid.items())
parameter_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

print(f"Total combinations to process: {len(parameter_combinations)}")


new_prompt_each_iteration = True

if not new_prompt_each_iteration:
    # Prompt override
    #prompt_override = 'A busy marketplace with cyborgs and humans trading goods and services, anime, 3d animated, 4k'
    prompt_override = None
    if prompt_override is None:
        prompt, prompt_filename = generate_prompt()
        print(f"Prompt: {prompt}")
    else:
        prompt = prompt_override
        prompt_filename = "prompt_override"
        print(f"Prompt: {prompt}")

# Message to encode in the QR code
#message = "https://github.com/tillzacher/driving_data_analysis.git"
message = "https://github.com/tillzacher"
#message = generate_wifi_qr_string("Vodafone-FD4C", "pxfy2x2gbq23p3hh")



# Loop over each combination and run the diffusion process
for idx, params in enumerate(parameter_combinations):
    print(f"Processing combination {idx + 1}/{len(parameter_combinations)}")

    if new_prompt_each_iteration:
        prompt, prompt_filename = generate_prompt()
        print(f"Prompt: {prompt}")

    # Unpack parameters
    strength = params['strength']
    guidance_scale = params['guidance_scale']
    controlnet_conditioning_scale = params['controlnet_conditioning_scale']
    resolution = params['resolution']
    seed = params['seed']
    mask_logo = params['mask_logo']
    border = params['border']
    noise_level = params['noise_level']
    border_noise_level = params['border_noise_level']
    center_noise_level = params['center_noise_level']
    num_steps = params['num_inference_steps']
    inverted = params['inverted']
    add_logo = params['add_logo']

    # Generate the QR code with the current 'inverted' parameter
    raw_qr_code = generate_qr_code(
        data=message,
        filename='latest_qr_pipeline',  # We're not saving the QR code image separately
        mask_logo=mask_logo,  # Adjust as needed
        border=border,
        inverted=inverted,
        add_logo=add_logo
    )

    # Generate a unique output filename for each combination
    output_filename = os.path.join(
        output_dir,
        f"{prompt_filename}_s{strength}_gs{guidance_scale}_ccs{controlnet_conditioning_scale}_r{resolution}_seed{seed}_nl{noise_level}_bnl{border_noise_level}_mask{mask_logo}_cnl{center_noise_level}_steps{num_steps}_inv{inverted}"
    )

    # Run the diffusion process
    run_diffusion_on_qr_code(
        image_of_qr=raw_qr_code,
        prompt=prompt,
        output_filename=output_filename,
        model_id='models/animerge_v23',  # Update this if you have a different model
        strength=strength,
        guidance_scale=guidance_scale,
        controlnet_conditioning_scale=controlnet_conditioning_scale,
        num_inference_steps=num_steps,
        resolution=resolution,
        seed=seed,
        noise_level=noise_level,
        border_noise_level=border_noise_level,
        center_noise_level=center_noise_level,
        mask_logo=mask_logo,
        inverted=inverted,
        verbose=False
    )

c:\Users\Schweini\miniconda3\envs\torch_diffusers_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Total combinations to process: 8
Processing combination 1/8
Prompt: A QR code, cleverly crafted into a colorful shopping mall, gothic theme, surrealism, 3D rendered
Error creating external output directory 'C:\Users\Napi\iCloudDrive\generated_qr_codes\large_files_low_border': [WinError 5] Zugriff verweigert: 'C:\\Users\\Napi'
Processing combination 2/8
Prompt: A QR code, cleverly incorporated into a dynamic fantasy castle, retro theme, vector illustration, 3D rendered
Error creating external output directory 'C:\Users\Napi\iCloudDrive\generated_qr_codes\large_files_low_border': [WinError 5] Zugriff verweigert: 'C:\\Users\\Napi'
Processing combination 3/8
Prompt: A QR code, cleverly crafted into a abstract ice cave, modern theme, oil painting, 3D rendered
Error creating external output directory 'C:\Users\Napi\iCloudDrive\generated_qr_codes\large_files_low_border': [WinError 5] Zugriff verweigert: 'C:\\Users\\Napi'
Processing combination 4/8
Prompt: A QR code, cleverly surrounded into a

In [6]:
# Print a message when all combinations have been processed
print("All combinations have been processed.")

All combinations have been processed.


In [ ]:
# define function to di